# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
#path to file
city_csvfile = "../output_data/city_data_df.csv"

#read file
city_df =pd.read_csv(city_csvfile)

#print
city_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# getting info from API key
gmaps.configure(api_key=g_key)
fig map = gmaps.figure()

#set idea conditions (max temp 70 to 80, wind speed less than 10 and zero clouds)
# https link
city_coordinates = city_df[['Lat', 'Lng']].astype(float)

#idea_cond_df = main_df
#print coordinates
#city_coordinates

# make humidity list 
humidity_list = city_df['Humidity'].astype(int)

In [ ]:
#setup humity heatmap layer
fig_map - gmaps.figure(map_type = 'TERRAIN')

#setup locations
heatmap_layer = gmaps.heatmap_layer(lacations, weights=humdidity_list, dissipating=False, max_intensity=100, point_radius = 2)

#add layers
fig_map.add_layer(heatmap_layer)

#print humidity list
humd_list

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#drop cities with a max temperature lower than 80 degrees but higher than 70.
cities_df = cities_df.drop(cities_df[(cities_df['Max Temp']>=80)].index)
cities_df = cities_df.drop(cities_df[(cities_df['Max Temp']<=70)].index)

#drop cities with wind speed less than 10 mph.
cities_df = cities_df.drop(cities_df[(cities_df['Wind Speed']>=10)].index)

#drop cities with cloudiness.
hotel_df = cities_df.drop(cities_df[(cities_df['Cloudiness']>0)].index)

#print results
hotel_df

In [ ]:
#setup city list
city_list = list(hotel_df['City'])

#Print
city_list

In [ ]:
#params dictionary
params = {'radius' :5000,
         'keyword': 'lodging',
         'key' : g_key}

#for loop to find hotel info for the ideal conditions on dataframe
for index, row in hotel_df.iterrows():
    
    #getting Lat and Lng from df
    lat = row['Lat']
    lng = row['Lng']
    
    #change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    #getting url
    url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #(params ['key'])
    response = requests.get(url, params=params)
    
    #convert to json
    results = response.json()
    
    #hotels = results['results']
    try:
        print(f"{results['results'][0]['name']} is the closest to lodging.")
        hotel_df.loc[index, 'Hotel_Name'] = results['results'][0]['name']
        
    except (KeyError, IndexError):
        
        #print
        print("Error found skipping")

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#Drop unwanted hotels
hotel_df = hotel_df.drop(hotel_df[(hotel_df['Hotel_Name'] == "")].index)

#print
hotel_df

In [ ]:
#for loop to find hotel info for the idea conditions on dataframe
for index, row in idea_cond_df.inerrows():
    lat = row['Lat']
    lng = row['Lng']
    
    #change location each iteration while leaving original params in place
    parans['location'] =f"{lat},{lng}"
    
    #getting url
    url ="http://maps.googleapis.com/map/api/place/nearbysearch/json"
    
    #(params ['key'])
    results = requests.get(url, params = parmas).json()
    hotels = results['results']
    
     #condition = results['results']
    try:
        print(f"{hotels[0]['name']} is the closest to lodging.")
        idea_cond_df.loc[index, 'Hotel Name'] = hotels[0]['name']
        
    except (KeyError, IndexError):
        print("Error found skipping")
#print       
hotel_df = idea_cond_df
hotel_df.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add hotel marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig_map.add_layer(marker_layer)

# Display figure
fig_map